In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
def setup_download_from_s3( rel_fp, recursive=True ):
    s3_fp = 's3://mousebrainatlas-data/'+rel_fp
    local_fp = os.environ['ROOT_DIR']+rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return 
    
    if recursive:
        ! aws s3 cp --recursive $s3_fp $local_fp
    else:
        ! aws s3 cp $s3_fp $local_fp

In [24]:
setup_download_from_s3('CSHL_patch_samples_features_V0/')

download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/4N/MD589_4N_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD589/4N/MD589_4N_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/4N/MD589_4N_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD589/4N/MD589_4N_negative.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/3N/MD589_3N_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD589/3N/MD589_3N_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/6N/MD589_6N_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD589/6N/MD589_6N_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/3N/MD589_3N_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features

download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/SC/MD589_SC_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD589/SC/MD589_SC_negative.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/Sp5C/MD589_Sp5C_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD589/Sp5C/MD589_Sp5C_negative.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/Sp5I/MD589_Sp5I_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD589/Sp5I/MD589_Sp5I_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/SNR/MD589_SNR_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD589/SNR/MD589_SNR_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD589/Sp5O/MD589_Sp5O_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSH

download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD594/LC/MD594_LC_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD594/LC/MD594_LC_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD594/PBG/MD594_PBG_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD594/PBG/MD594_PBG_negative.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD594/DC/MD594_DC_positive.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD594/DC/MD594_DC_positive.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD594/LRt/MD594_LRt_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features_V0/MD594/LRt/MD594_LRt_negative.pkl
download: s3://mousebrainatlas-data/CSHL_patch_samples_features_V0/MD594/RMC/MD594_RMC_negative.pkl to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_sample

In [41]:
setup_download_from_s3('CSHL_hsv/MD589/')

download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_120.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_120.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_117.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_117.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_116.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_116.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_115.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_115.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_122.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_122.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_119.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_119.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_121.tif to ../../../../BstemAtlasDataBackup/u

download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_311.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_311.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_319.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_319.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_320.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_320.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_325.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_325.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_329.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_329.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_318.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_hsv/MD589/Sp5I/Sp5I_318.tif
download: s3://mousebrainatlas-data/CSHL_hsv/MD589/Sp5I/Sp5I_327.tif to ../../../../BstemAtlasDataBackup/u

In [3]:
#credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
credFiles= '/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [4]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [22]:
stack = 'MD589'
yaml_file = 'shape_params-aws.yaml'
#yaml_file = 'shape_params.yaml'
patch_file = 'CSHL_patches'
scripts_dir = os.environ['REPO_DIR']

In [5]:
Structure6.drop()

`kui_diffusionmap`.`structure6` (28 tuples)
`kui_diffusionmap`.`__process6` (28 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [6]:
@schema
class Structure6(dj.Manual):
    definition = """
    id  : int   # sequence
    --------
    structure       : char(10)    # name of mouse brain's structure
    positive_s3_fp  : varchar(200)  # (str) file path
    negative_s3_fp  : varchar(200)  # (str) file path
    """

In [7]:
paired_structures = ['5N', '6N', '7n', '7N', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC'] #
all_structures = paired_structures + singular_structures
s3_fp = patch_file+'_features/'+stack+'/' #s3://mousebrainatlas-data/
print(s3_fp)

CSHL_patches_features/MD589/


In [8]:
n=0
for struc in all_structures:
    print("\nAdding "+struc+' to the database')
    Structure6.insert1(dict(id=n,
                           structure=struc,
                           positive_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_positive.pkl',
                           negative_s3_fp=s3_fp+struc+'/'+stack+'_'+struc+'_negative.pkl',)
                      ,skip_duplicates=False)
    n+=1


Adding 5N to the database

Adding 6N to the database

Adding 7n to the database

Adding 7N to the database

Adding Amb to the database

Adding LC to the database

Adding LRt to the database

Adding Pn to the database

Adding Tz to the database

Adding VLL to the database

Adding RMC to the database

Adding SNC to the database

Adding SNR to the database

Adding 3N to the database

Adding 4N to the database

Adding Sp5I to the database

Adding Sp5O to the database

Adding Sp5C to the database

Adding PBG to the database

Adding 10N to the database

Adding VCA to the database

Adding VCP to the database

Adding DC to the database

Adding AP to the database

Adding 12N to the database

Adding RtTg to the database

Adding SC to the database

Adding IC to the database


In [9]:
structureTable = Structure6.fetch(as_dict=True)
strucDF = pd.DataFrame(structureTable)
strucDF

,id,structure,positive_s3_fp,negative_s3_fp
0,0,5N,CSHL_patches_features/MD589/5N/MD589_5N_positi...,CSHL_patches_features/MD589/5N/MD589_5N_negati...
1,1,6N,CSHL_patches_features/MD589/6N/MD589_6N_positi...,CSHL_patches_features/MD589/6N/MD589_6N_negati...
2,2,7n,CSHL_patches_features/MD589/7n/MD589_7n_positi...,CSHL_patches_features/MD589/7n/MD589_7n_negati...
3,3,7N,CSHL_patches_features/MD589/7N/MD589_7N_positi...,CSHL_patches_features/MD589/7N/MD589_7N_negati...
4,4,Amb,CSHL_patches_features/MD589/Amb/MD589_Amb_posi...,CSHL_patches_features/MD589/Amb/MD589_Amb_nega...
5,5,LC,CSHL_patches_features/MD589/LC/MD589_LC_positi...,CSHL_patches_features/MD589/LC/MD589_LC_negati...
6,6,LRt,CSHL_patches_features/MD589/LRt/MD589_LRt_posi...,CSHL_patches_features/MD589/LRt/MD589_LRt_nega...
7,7,Pn,CSHL_patches_features/MD589/Pn/MD589_Pn_positi...,CSHL_patches_features/MD589/Pn/MD589_Pn_negati...
8,8,Tz,CSHL_patches_features/MD589/Tz/MD589_Tz_positi...,CSHL_patches_features/MD589/Tz/MD589_Tz_negati...
9,9,VLL,CSHL_patches_features/MD589/VLL/MD589_VLL_posi...,CSHL_patches_features/MD589/VLL/MD589_VLL_nega...


In [10]:
Process6.drop()

NetworkXError: The node `kui_diffusionmap`.`__process6` is not in the graph.

In [11]:
@schema
class Process6(dj.Computed):
    definition="""
    -> Structure6
    -----
    size_positive_properties : int   #size of file
    size_negative_properties : int   #size of file
    """

    bucket = "mousebrainatlas-data"
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure6 & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        for state in ['positive', 'negative']:
            item_name = state+'_s3_fp'
            key_item = 'size_'+state+'_properties'
            s3_fp = (Structure6 & key).fetch1(item_name)
            try:
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size/1000)
            except:
                ! python3 $scripts_dir/Cell_generator.py $stack $struc $state $yaml_file $patch_file
                upload_fp = 's3://'+self.bucket+'/'+s3_fp
                pkl_out_file = os.environ['ROOT_DIR']+s3_fp
                ! aws s3 cp $pkl_out_file $upload_fp
                print(pkl_out_file,upload_fp)
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size / 1000)
        try:
            self.insert1(key)
        except:
            print('could not insert key=', key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [12]:
Process6.populate()

populating for  5N
2019-07-02 05:30:55,731	WARNING worker.py:1331 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-07-02 05:30:55,734	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-02_05-30-55_731741_19256/logs.
2019-07-02 05:30:55,841	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:17987 to respond...
2019-07-02 05:30:55,949	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:31374 to respond...
2019-07-02 05:30:55,950	INFO services.py:806 -- Starting Redis shard with 10.0 GB max memory.
2019-07-02 05:30:55,961	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-02_05-30-55_731741_19256/logs.
2019-07-02 05:30:55,961	WARNING services.py:1294 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_mem

populating for  6N
2019-07-02 05:36:36,703	WARNING worker.py:1331 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-07-02 05:36:36,703	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-02_05-36-36_703464_18819/logs.
2019-07-02 05:36:36,809	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:10425 to respond...
2019-07-02 05:36:36,918	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:61527 to respond...
2019-07-02 05:36:36,919	INFO services.py:806 -- Starting Redis shard with 10.0 GB max memory.
2019-07-02 05:36:36,930	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-02_05-36-36_703464_18819/logs.
2019-07-02 05:36:36,931	WARNING services.py:1294 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_mem

(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/162_7.tif std= 9.208101225745391 too blank
(pid=35510) 7n_positive 81 80 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/253_5.tif std= 8.801635762895184 too blank
(pid=35510) 7n_positive 91 90 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/153_0.tif std= 8.238259619117889 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/163_5.tif std= 9.541213693371379 too blank
(pid=35510) 7n_positive 101 100 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/284_9.tif std= 9.682229809977786 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/154_5.tif std= 6.706864757772185 too blank
(pid=35510) 7n_positive 111 110 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/313_4.tif std= 8.893286338468352 too blank
(

(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/161_8.tif std= 9.225603082595327 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/164_6.tif std= 9.365153556112809 too blank
(pid=35510) 7n_positive 311 310 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/188_9.tif std= 9.804948101526024 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/170_6.tif std= 6.58209334901853 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/310_7.tif std= 8.704716783452007 too blank
(pid=35510) 7n_positive 321 320 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/201_2.tif std= 9.896569333511557 too blank
(pid=35510) 7n_positive 331 330 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/174_3.tif std= 8.130816638469247 too blank
(pid=35510) image /data/BstemAtlasDat

(pid=35510) 7n_positive 561 560 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/185_2.tif std= 9.832832817037731 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/159_5.tif std= 8.878813314660235 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/198_5.tif std= 8.650959893336221 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/164_7.tif std= 8.688396240163826 too blank
(pid=35510) 7n_positive 571 570 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/268_5.tif std= 8.918063884683823 too blank
(pid=35510) 7n_positive 581 580 / 1000
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/158_0.tif std= 4.927022214544689 too blank
(pid=35510) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/7n/176_9.tif std= 6.44385492079012 too blank
(pid=35510) 7n_positive 591 590 / 100

...

(pid=14200) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/12N_surround_500um_noclass/224_7.tif std= 3.8278148701790946 too blank
(pid=14200) 12N_negative 151 150 / 390
(pid=14200) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/12N_surround_500um_noclass/228_7.tif std= 3.2077654422163127 too blank
(pid=14200) 12N_negative 161 160 / 390
(pid=14200) 12N_negative 171 170 / 390
(pid=14200) 12N_negative 181 180 / 390
(pid=14200) 12N_negative 191 190 / 390
(pid=14200) 12N_negative 201 200 / 390
(pid=14200) 12N_negative 211 210 / 390
(pid=14200) 12N_negative 220 220 / 390
(pid=14200) 12N_negative 230 230 / 390
(pid=14200) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/12N_surround_500um_noclass/231_3.tif std= 9.10487413230216 too blank
(pid=14200) 12N_negative 240 240 / 390
(pid=14200) 12N_negative 250 250 / 390
(pid=14200) 12N_negative 260 260 / 390
(pid=14200) 12N_negative 270 270 / 390
(pid=14200) 12N_negative 280 280 / 390
(pid=14200) 12N_ne

/data/Github/shapeology_code/scripts/lib/utils.py:22: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.D=yaml.load(open(yamlFile,'r'))
/data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches_features/MD589/
(pid=27024) SC_positive 1 0 / 1000
(pid=27024) SC_positive 11 10 / 1000
(pid=27024) SC_positive 21 20 / 1000
(pid=27024) SC_positive 31 30 / 1000
(pid=27024) SC_positive 41 40 / 1000
(pid=27024) SC_positive 51 50 / 1000
(pid=27024) SC_positive 61 60 / 1000
(pid=27024) SC_positive 71 70 / 1000
(pid=27024) SC_positive 81 80 / 1000
(pid=27024) SC_positive 91 90 / 1000
(pid=27024) SC_positive 101 100 / 1000
(pid=27024) SC_positive 111 110 / 1000
(pid=27024) SC_positive 121 120 / 1000
(pid=27024) SC_positive 131 130 / 1000
(pid=27024) SC_positive 141 140 / 1000
(pid=27024) SC_positive 151 150 / 1000
(pid=27024) SC_positive 161 160 / 1000
(pid=27024) SC_positive 171 170 / 100

(pid=4203) SC_negative 101 100 / 1000
(pid=4203) SC_negative 111 110 / 1000
(pid=4203) SC_negative 121 120 / 1000
(pid=4203) SC_negative 131 130 / 1000
(pid=4203) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/SC_surround_500um_noclass/220_9.tif std= 3.09177374716207 too blank
(pid=4203) SC_negative 141 140 / 1000
(pid=4203) SC_negative 151 150 / 1000
(pid=4203) SC_negative 161 160 / 1000
(pid=4203) SC_negative 171 170 / 1000
(pid=4203) SC_negative 181 180 / 1000
(pid=4203) SC_negative 191 190 / 1000
(pid=4203) SC_negative 201 200 / 1000
(pid=4203) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/SC_surround_500um_noclass/245_7.tif std= 5.050526794105165 too blank
(pid=4203) SC_negative 211 210 / 1000
(pid=4203) SC_negative 221 220 / 1000
(pid=4203) SC_negative 231 230 / 1000
(pid=4203) SC_negative 241 240 / 1000
(pid=4203) SC_negative 251 250 / 1000
(pid=4203) SC_negative 261 260 / 1000
(pid=4203) SC_negative 271 270 / 1000
(pid=4203) image /data/BstemAtl

(pid=13296) IC_positive 1 0 / 1000
(pid=13296) IC_positive 11 10 / 1000
(pid=13296) IC_positive 21 20 / 1000
(pid=13296) IC_positive 31 30 / 1000
(pid=13296) IC_positive 41 40 / 1000
(pid=13296) IC_positive 51 50 / 1000
(pid=13296) IC_positive 61 60 / 1000
(pid=13296) IC_positive 71 70 / 1000
(pid=13296) IC_positive 81 80 / 1000
(pid=13296) IC_positive 91 90 / 1000
(pid=13296) IC_positive 101 100 / 1000
(pid=13296) IC_positive 111 110 / 1000
(pid=13296) IC_positive 121 120 / 1000
(pid=13296) IC_positive 131 130 / 1000
(pid=13296) IC_positive 141 140 / 1000
(pid=13296) IC_positive 151 150 / 1000
(pid=13296) IC_positive 161 160 / 1000
(pid=13296) IC_positive 171 170 / 1000
(pid=13296) IC_positive 181 180 / 1000
(pid=13296) IC_positive 191 190 / 1000
(pid=13296) IC_positive 201 200 / 1000
(pid=13296) IC_positive 211 210 / 1000
(pid=13296) IC_positive 221 220 / 1000
(pid=13296) IC_positive 231 230 / 1000
(pid=13296) IC_positive 241 240 / 1000
(pid=13296) IC_positive 251 250 / 1000
(pid=132

(pid=28579) IC_negative 141 140 / 1000
(pid=28579) IC_negative 151 150 / 1000
(pid=28579) IC_negative 161 160 / 1000
(pid=28579) IC_negative 171 170 / 1000
(pid=28579) IC_negative 181 180 / 1000
(pid=28579) IC_negative 191 190 / 1000
(pid=28579) IC_negative 201 200 / 1000
(pid=28579) IC_negative 211 210 / 1000
(pid=28579) IC_negative 221 220 / 1000
(pid=28579) image /data/BstemAtlasDataBackup/ucsd_brain/CSHL_patches/MD589/IC_surround_500um_noclass/230_4.tif std= 3.974346830543976 too blank
(pid=28579) IC_negative 231 230 / 1000
(pid=28579) IC_negative 241 240 / 1000
(pid=28579) IC_negative 251 250 / 1000
(pid=28579) IC_negative 261 260 / 1000
(pid=28579) IC_negative 271 270 / 1000
(pid=28579) IC_negative 281 280 / 1000
(pid=28579) IC_negative 291 290 / 1000
(pid=28579) IC_negative 301 300 / 1000
(pid=28579) IC_negative 311 310 / 1000
(pid=28579) IC_negative 321 320 / 1000
(pid=28579) IC_negative 331 330 / 1000
(pid=28579) IC_negative 341 340 / 1000
(pid=28579) IC_negative 350 350 / 100

In [5]:
Structure.fetch(as_dict=True)

[OrderedDict([('id', 0), ('structure', '5N')]),
 OrderedDict([('id', 1), ('structure', '6N')]),
 OrderedDict([('id', 2), ('structure', '7nn')]),
 OrderedDict([('id', 3), ('structure', '7N')]),
 OrderedDict([('id', 4), ('structure', 'Amb')]),
 OrderedDict([('id', 5), ('structure', 'LC')]),
 OrderedDict([('id', 6), ('structure', 'LRt')]),
 OrderedDict([('id', 7), ('structure', 'Pn')]),
 OrderedDict([('id', 8), ('structure', 'Tz')]),
 OrderedDict([('id', 9), ('structure', 'VLL')]),
 OrderedDict([('id', 10), ('structure', 'RMC')]),
 OrderedDict([('id', 11), ('structure', 'SNC')]),
 OrderedDict([('id', 12), ('structure', 'SNR')]),
 OrderedDict([('id', 13), ('structure', '3N')]),
 OrderedDict([('id', 14), ('structure', '4N')]),
 OrderedDict([('id', 15), ('structure', 'Sp5I')]),
 OrderedDict([('id', 16), ('structure', 'Sp5O')]),
 OrderedDict([('id', 17), ('structure', 'Sp5C')]),
 OrderedDict([('id', 18), ('structure', 'PBG')]),
 OrderedDict([('id', 19), ('structure', '10N')]),
 OrderedDict([(

In [23]:
Process.fetch(as_dict=True)

[OrderedDict([('id', 0), ('image_number', 44)]),
 OrderedDict([('id', 1), ('image_number', 5)]),
 OrderedDict([('id', 2), ('image_number', 125)]),
 OrderedDict([('id', 3), ('image_number', 89)]),
 OrderedDict([('id', 4), ('image_number', 13)]),
 OrderedDict([('id', 5), ('image_number', 24)]),
 OrderedDict([('id', 6), ('image_number', 76)]),
 OrderedDict([('id', 8), ('image_number', 50)]),
 OrderedDict([('id', 9), ('image_number', 69)]),
 OrderedDict([('id', 10), ('image_number', 57)]),
 OrderedDict([('id', 11), ('image_number', 80)]),
 OrderedDict([('id', 12), ('image_number', 126)]),
 OrderedDict([('id', 13), ('image_number', 24)]),
 OrderedDict([('id', 14), ('image_number', 11)]),
 OrderedDict([('id', 15), ('image_number', 73)]),
 OrderedDict([('id', 16), ('image_number', 67)]),
 OrderedDict([('id', 18), ('image_number', 24)]),
 OrderedDict([('id', 19), ('image_number', 45)]),
 OrderedDict([('id', 20), ('image_number', 65)]),
 OrderedDict([('id', 21), ('image_number', 49)])]

In [7]:
import pickle
fname = os.path.join(os.environ['ROOT_DIR'], 'CSHL_data_processed', stack, 'All_patch_locations.pkl')
all_patch_locations = pickle.load(open(fname, 'rb'), encoding='latin1')

In [29]:
len(all_patch_locations['SC'].keys())

181

In [1]:
! python ../Cell_generator.py 'MD589' '4N' 'positive' 'shape_params.yaml' 'CSHL_patch_samples'

/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-07-25 04:16:26,851	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-25_04-16-26_5765/logs.
2019-07-25 04:16:26,966	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:25068 to respond...
2019-07-25 04:16:27,188	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:33464 to respond...
2019-07-25 04:16:27,191	INFO services.py:804 -- Starting Redis shard with 1.72 GB max memory.
2019-07-25 04:16:27,202	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-25_04-16-26_5765/logs.
2019-07-25 04:16:27,203	INFO services.py:1427 -- Starting the Plasma object store with 2.58 GB memory using /tmp.
/Users/kuiqian/Github/s

(pid=5779) Completed 812.4 KiB/1.6 MiB (2.1 MiB/s) with 24 file(s) remaining
(pid=5779) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/4N/234_1.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/4N/234_1.tif
(pid=5779) Completed 812.4 KiB/1.6 MiB (2.1 MiB/s) with 23 file(s) remaining
(pid=5779) Completed 850.6 KiB/1.6 MiB (2.2 MiB/s) with 23 file(s) remaining
(pid=5779) Completed 885.2 KiB/1.6 MiB (2.3 MiB/s) with 23 file(s) remaining
(pid=5779) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/4N/235_0.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/4N/235_0.tif
(pid=5779) Completed 885.2 KiB/1.6 MiB (2.3 MiB/s) with 22 file(s) remaining
(pid=5779) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/4N/235_1.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/4N/235_1.tif
(pid=5779) Completed 885.2 KiB/1.6 MiB (2.3 MiB/s) with 21 file(s) remaining
(pid=5779) Completed 919.9 K

(pid=5779) Completed 1.6 MiB/1.6 MiB (2.9 MiB/s) with 1 file(s) remaining
(pid=5779) download: s3://mousebrainatlas-data/CSHL_patch_samples/MD589/4N/240_0.tif to ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples/MD589/4N/240_0.tif
(pid=5779) 4N_positive 1 0 / 45
(pid=5779) 4N_positive 11 10 / 45
(pid=5779) 4N_positive 21 20 / 45
(pid=5779) 4N_positive 31 30 / 45
(pid=5779) 4N_positive 41 40 / 45
(pid=5779) 4N_positive 45
(pid=5779) run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD589/4N/MD589_4N_positive.pkl s3://mousebrainatlas-data/CSHL_patch_samples_features/MD589/4N/MD589_4N_positive.pkl
(pid=5779) Completed 256.0 KiB/272.3 KiB (585.8 KiB/s) with 1 file(s) remaining
(pid=5779) Completed 272.3 KiB/272.3 KiB (380.0 KiB/s) with 1 file(s) remaining
(pid=5779) upload: ../../../../BstemAtlasDataBackup/ucsd_brain/CSHL_patch_samples_features/MD589/4N/MD589_4N_positive.pkl to s3://mousebrainatlas-data/CSHL_patch_samples_features/MD

In [30]:
schema.jobs

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__sample_features,647b2a336fa813f6b4bee8e561b090c4,reserved,=BLOB=,,=BLOB=,kui@172.31.21.247,ip-172-31-21-247,8324,24204,2019-07-29 03:32:50
__sample_features,66e96423819dd99274f4a2c2ebc02f61,reserved,=BLOB=,,=BLOB=,kui@172.31.21.252,ip-172-31-21-252,8274,24203,2019-07-29 03:32:11
__sample_features,6a7698388ce6434a04602d72ed1eb955,reserved,=BLOB=,,=BLOB=,kui@172.31.16.51,ip-172-31-16-51,8322,24196,2019-07-29 03:26:04
__sample_features,71a8a02dc1c7e9911c21fdc5bba63db0,reserved,=BLOB=,,=BLOB=,kui@172.31.31.54,ip-172-31-31-54,8310,24200,2019-07-29 03:29:53
__sample_features,8eee1505bf48577ff9c0274e241f0da6,reserved,=BLOB=,,=BLOB=,kui@172.31.30.49,ip-172-31-30-49,8284,24195,2019-07-29 03:30:02
__sample_features,d3d85f072985af3dd2cc517706390cc1,reserved,=BLOB=,,=BLOB=,kui@172.31.25.59,ip-172-31-25-59,8271,24202,2019-07-29 03:31:18


In [31]:
SampleFeatures.fetch(as_dict=True)

[OrderedDict([('structure', '10N'),
              ('size_of_positive', 620),
              ('size_of_negative', 700)]),
 OrderedDict([('structure', '12N'),
              ('size_of_positive', 596),
              ('size_of_negative', 585)]),
 OrderedDict([('structure', '3N'),
              ('size_of_positive', 367),
              ('size_of_negative', 367)]),
 OrderedDict([('structure', '4N'),
              ('size_of_positive', 68),
              ('size_of_negative', 183)]),
 OrderedDict([('structure', '5N'),
              ('size_of_positive', 673),
              ('size_of_negative', 672)]),
 OrderedDict([('structure', '6N'),
              ('size_of_positive', 65),
              ('size_of_negative', 75)]),
 OrderedDict([('structure', '7N'),
              ('size_of_positive', 1353),
              ('size_of_negative', 1361)]),
 OrderedDict([('structure', '7nn'),
              ('size_of_positive', 1303),
              ('size_of_negative', 1529)]),
 OrderedDict([('structure', 'Amb'),
        

In [11]:
(schema.jobs & 'status="error"' ).delete()

In [27]:
SampleFeatures.drop()

`kui_diffusionmap`.`__sample_features` (28 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.
